In [4]:
# 07-02. CNN으로 MNIST 분류하기 

# 1) 필요한 도구 임포트와 입력 정의 
import torch 
import torch.nn as nn

# 배치 크기 * 채널 * 높이(height) * 너비(width) 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기: {}'.format(inputs.shape))

텐서의 크기: torch.Size([1, 1, 28, 28])


In [5]:
# 2) 합성곱층과 풀링 선언하기 
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding =1)
print(conv2)

pool = nn.MaxPool2d(2)
print(pool)

# 3) 구현체를 연결하여 모델 만들기 
out = conv1(inputs)
print(out.shape)

out = pool(out)
print(out.shape)

out = conv2(out)
print(out.shape)

out = pool(out)
print(out.shape)

# 4) 텐서를 펼치는 작업 
print(out.size(0))  # out의 첫번째 차원 = 1
print(out.size(1))  # out의 두번째 차원 = 64
print(out.size(2))  # out의 세번쨰 차원 = 7
print(out.size(3))  # out의 네번째 차원 = 7

# 첫번째 차원인 배치 차원은 그대로 두고 view()를 이용하여 나머지는 펼쳐라 
# ex) reshape() = view()
out = out.view(out.size(0), -1)
print(out.shape)    # [1, 3136]

# 전결합층(Fully-Connected layer) 선언
fc = nn.Linear(3136, 10)    # input_dim = 3136, output_dim = 10

# 구현체를 연결하여 모델 만들기 
out = fc(out)
print(out.shape)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
1
64
7
7
torch.Size([1, 3136])
torch.Size([1, 10])


In [14]:
import torch 
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda': 
    torch.cuda.manual_seed_all(777)

# 학습에 사용할 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터셋 정의 
mnist_train = dsets.MNIST(root = 'MNIST_data\\', 
                          train=True, 
                          transform = transforms.ToTensor(), 
                          download = True
                          )

mnist_test = dsets.MNIST(root = 'MNIST_data\\',  # 다운로드 경로 지정 
                         train = False,         # False를 지정하면 테스트 데이터로 다운로드 
                         transform = transforms.ToTensor(), # 텐서로 변환
                         download = True)

# 데이터로더를 사용하여 데이터 다루기 
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size = batch_size, 
                                          shuffle = True, 
                                          drop_last = True)

class CNN(torch.nn.Module): 
    def __init__(self): 
        super(CNN, self).__init__()
        # 첫번째층 
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1), 
            torch.nn.ReLU(), 
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        # 두번째층 
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1), 
            torch.nn.ReLU(), 
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # 전결합층 
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)

        # 전결합층 한정으로 가중치 초기화 
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x): 
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 전결합층을 위해서 Flatten 
        out = self.fc(out)
        return out

# CNN 모델 정의 
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)  # 비용 함수에 소프트맥스 함수 포함되어져 있음 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

for epoch in range(training_epochs): 
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블. 
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch 
    
    print('[Epoch: {:>4}] cost = {:>9.}'.format(epoch + 1, avg_cost))


# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad(): 
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    acccuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

총 배치의 수: 600


ValueError: Format specifier missing precision